### import

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

### 함수 : 상품명 전처리

In [ ]:
# prompt: 파라미터로 전달되는 상품명에 대해 앞위 공백 제거 후 끝 문자가 g 또는 G 인 경우 바로 앞 공백까지 제거하여 반환하는 함수

def preprocess_product_name(product_name):
    """
    상품명에 대한 전처리 함수.

    Args:
        product_name: 전처리할 상품명.

    Returns:
        전처리된 상품명.
    """
    product_name = product_name.strip()  # 앞뒤 공백 제거
    if product_name and (product_name[-1] == 'g' or product_name[-1] == 'G'):
        # 끝 문자가 g 또는 G 이면서 상품명이 비어있지 않은 경우
        for i in range(len(product_name) - 2, -1, -1):
            if product_name[i] == ' ':
                product_name = product_name[:i]
                break
        else:
            product_name = product_name[:-1]  # 공백 없으면 g 또는 G만 제거
    return product_name

### 함수 : 상품 목록 수집

In [ ]:
def get_product_list(get_cnt):
    url = "https://emart.ssg.com/disp/ajaxCategory.ssg"
    params = {
        "dispCtgId": "6000217707",
        "sort": "sale",
        "page": 1
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    response = requests.get(url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    products = []
    items = soup.find_all('li', class_='mnemitem_grid_item')[:get_cnt]

    for item in items:
        product_id = item.select_one('div.mnemitem_unit')['data-react-unit-id']
        product_nm_tmp = item.select_one('span.mnemitem_goods_tit').text.strip()
        product_name = item.find('span', class_='mnemitem_goods_tit').text.strip()
        product_nm = preprocess_product_name(product_nm_tmp)
        products.append({'product_id': product_id, 'product_name': product_name})

    return products

### 함수 : 리뷰 수집

In [ ]:
def get_reviews(product_id):
    print(f"상품 '{product_id}'의 리뷰 수집....")

    reviews = []
    page = 1

    while True:
        url = "https://emart.ssg.com/item/ajaxItemCommentList.ssg"
        params = {
            "itemId": product_id,
            "siteNo": "7009",
            "page": page,
            "pageSize": 10
        }
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
            'Accept': 'text/html, */*; q=0.01',
            'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
            'Referer': f'https://emart.ssg.com/item/itemView.ssg?itemId={product_id}&siteNo=7009&salestrNo=2493'
        }

        response = requests.get(url, params=params, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        review_items = soup.select('li.rvw_expansion_panel.v2')
        if not review_items:
            print(f"상품 '{product_id}'의 모든 리뷰를 가져왔습니다.")
            break

        for item in review_items:
            try:
                rating = item.select_one('span.blind em').text
                content = item.select_one('p.rvw_item_text').text.strip()
                date = item.select_one('div.rvw_item_date').text.strip()

                reviews.append({
                    'rating': int(rating),
                    'content': content,
                    'date': date
                })
            except:
                continue

        page += 1
        time.sleep(1)  # 서버 부하 방지

    return reviews

### 실행

In [ ]:
def main():
    # 전체 데이터를 저장할 리스트
    all_data = []

    # 상품 목록 가져오기
    products = get_product_list(20)

    rank = 1

    # 각 상품별 리뷰 수집
    for product in products:
        product_id = product['product_id']
        product_name = product['product_name']

        print(f"상품ID: '{product_id}', 상품명: '{product_name}' 리뷰 수집 시작")

        reviews = get_reviews(product_id)

        # 각 리뷰에 상품 정보 추가
        for review in reviews:
            review['product_id'] = product_id
            review['product_name'] = product_name
            review['rank'] = rank
            all_data.append(review)

        print(f"상품 '{product_name}' 리뷰 수집 완료 ({len(reviews)}개)")
        rank += 1
        time.sleep(2)  # 서버 부하 방지를 위한 지연

    # DataFrame 생성 및 CSV 저장
    df = pd.DataFrame(all_data)
    current_time = datetime.now().strftime("%Y%m%d_%H%M")
    df.to_csv(f'emart_reviews_{current_time}.csv', index=False, encoding='utf-8-sig')
    print(f"전체 {len(all_data)}개의 리뷰 수집 완료")

if __name__ == "__main__":
    main()

상품ID: '1000604385774', 상품명: '송탄식 부대찌개 1.538kg' 리뷰 수집 시작
상품 '1000604385774'의 리뷰 수집....
상품 '1000604385774'의 모든 리뷰를 가져왔습니다.
상품 '송탄식 부대찌개 1.538kg' 리뷰 수집 완료 (1223개)
상품ID: '1000575098562', 상품명: '샤브샤브 요리재료 870g' 리뷰 수집 시작
상품 '1000575098562'의 리뷰 수집....
상품 '1000575098562'의 모든 리뷰를 가져왔습니다.
상품 '샤브샤브 요리재료 870g' 리뷰 수집 완료 (2032개)
상품ID: '1000595812799', 상품명: '강릉식 짬뽕순두부 밀키트 1.035 kg' 리뷰 수집 시작
상품 '1000595812799'의 리뷰 수집....
상품 '1000595812799'의 모든 리뷰를 가져왔습니다.
상품 '강릉식 짬뽕순두부 밀키트 1.035 kg' 리뷰 수집 완료 (625개)
상품ID: '1000599436903', 상품명: '햄폭탄 부대찌개 798g' 리뷰 수집 시작
상품 '1000599436903'의 리뷰 수집....
상품 '1000599436903'의 모든 리뷰를 가져왔습니다.
상품 '햄폭탄 부대찌개 798g' 리뷰 수집 완료 (541개)
상품ID: '1000049359710', 상품명: '순두부찌개 요리재료 804g' 리뷰 수집 시작
상품 '1000049359710'의 리뷰 수집....
상품 '1000049359710'의 모든 리뷰를 가져왔습니다.
상품 '순두부찌개 요리재료 804g' 리뷰 수집 완료 (6475개)
상품ID: '1000552428426', 상품명: '우삼겹 순두부찌개615g' 리뷰 수집 시작
상품 '1000552428426'의 리뷰 수집....
상품 '1000552428426'의 모든 리뷰를 가져왔습니다.
상품 '우삼겹 순두부찌개615g' 리뷰 수집 완료 (4548개)
상품ID: '1000633879761', 상품명: '[SSG ONLY] 소시지듬뿍 의정